## Problem 1

Approach A: A simple appraoch to solve this problem is to create python class that filters out the rows using the pandas data frame. We can have sequential filtering or all the columns followed by union to solve this problem. It is fairly a simple idea and works well.
However it does not scales well with increasing amount of data. As more computational power is required fore more rows being added.

Approach B: The second method is to implement a Graph gased approach where the data would be represented in the form of nodes and they will be inter connected to each other. A popular database to implement this is through the usage of AuraDB (Neo4J). However it is quite complex to build such system and is time consuming to come out with correct attributes and the relationships between the nodes.

### Apprach 1 implementation: 

1. This is much simpler approach where we create the data frame out of the data first. Next we simpley filter out the required rows using the pandas functions.
2. At last we would merge the values.

In [291]:
import pandas as pd
from tabulate import tabulate


# Pretty print class to print data from data frame in a tabular format
# Pretty print class to print data from data frame in a tabular format
class PrettyPrint:

    @staticmethod
    def print_dataframe(data: dict, title: str):
        print(f"{title}")
        print(tabulate(data, headers="keys", tablefmt="grid"))

    @staticmethod
    def print_node_data(data: dict, title: str):
        temp_list_of_dict=[]
        for event in data:
            temp_dict={}
            for item in event:
                temp_dict[item]=event[item]
            temp_list_of_dict.append(temp_dict)
        dict_to_dataframe=pd.DataFrame.from_dict(temp_list_of_dict)
        PrettyPrint.print_dataframe(dict_to_dataframe, title)

In [292]:
import pandas as pd

""" 
This class is used to filter data from the dataframes based on the filters provided by the user.
"""


class DataFrameFilter:
    def __init__(
        self, events_df, attendees_df, companies_df, contacts_df, employees_df
    ):
        self.events_df = events_df
        self.attendees_df = attendees_df
        self.companies_df = companies_df
        self.contacts_df = contacts_df
        self.employees_df = employees_df

    def filter_events(self, event_filters):
        if event_filters:
            filtered_events = self.events_df
            for col, values in event_filters.items():
                filtered_events = filtered_events[filtered_events[col].isin(values)]
        else:
            filtered_events = self.events_df
        return filtered_events

    def filter_attendees(self, filtered_events):
        return self.attendees_df[
            self.attendees_df["event_url"].isin(filtered_events["event_url"])
        ]

    def filter_companies_by_attendees(self, filtered_attendees):
        return self.companies_df[
            self.companies_df["company_url"].isin(filtered_attendees["company_url"])
        ]

    def apply_company_filters(self, filtered_companies, company_filters):
        if company_filters:
            for col, values in company_filters.items():
                filtered_companies = filtered_companies[
                    filtered_companies[col].isin(values)
                ]
        return filtered_companies

    def filter_employees(self, filtered_companies):
        return self.employees_df[
            self.employees_df["company_url"].isin(filtered_companies["company_url"])
        ]

    def apply_person_filters(self, filtered_employees, person_filters):
        if person_filters:
            for col, values in person_filters.items():
                filtered_employees = filtered_employees[
                    filtered_employees[col].isin(values)
                ]
        return filtered_employees

    def filter_final_companies(self, filtered_companies, filtered_employees):
        return filtered_companies[
            filtered_companies["company_url"].isin(filtered_employees["company_url"])
        ]

    def filter_final_events(self, filtered_events, filtered_attendees):
        return filtered_events[
            filtered_events["event_url"].isin(filtered_attendees["event_url"])
        ]

    def filter_data(
        self, event_filters=None, company_filters=None, person_filters=None
    ):
        # Filter data step by step
        filtered_events = self.filter_events(event_filters)
        filtered_attendees = self.filter_attendees(filtered_events)
        filtered_companies = self.filter_companies_by_attendees(filtered_attendees)
        filtered_companies = self.apply_company_filters(
            filtered_companies, company_filters
        )
        filtered_employees = self.filter_employees(filtered_companies)
        filtered_employees = self.apply_person_filters(
            filtered_employees, person_filters
        )
        filtered_companies = self.filter_final_companies(
            filtered_companies, filtered_employees
        )
        filtered_attendees = self.filter_attendees(filtered_events)
        filtered_events = self.filter_final_events(filtered_events, filtered_attendees)

        return filtered_events, filtered_companies, filtered_employees

In [293]:
# Sample data for events_df
events_data = {
    "event_url": ["event_1", "event_2", "event_3"],
    "event_name": ["Tech Summit", "Oil & Gas Expo", "Healthcare Conference"],
    "event_start_date": ["2024-09-10", "2024-10-15", "2024-11-20"],
    "event_city": ["San Francisco", "Houston", "New York"],
    "event_country": ["USA", "USA", "USA"],
    "event_industry": ["Technology", "Oil & Gas", "Healthcare"],
}
events_df = pd.DataFrame(events_data)

# Sample data for attendees_df
attendees_data = {
    "event_url": ["event_1", "event_1", "event_2", "event_2", "event_3"],
    "company_url": ["company_1", "company_2", "company_3", "company_4", "company_5"],
    "company_relation_to_event": [
        "Sponsor",
        "Exhibitor",
        "Sponsor",
        "Speaker",
        "Attendee",
    ],
}
attendees_df = pd.DataFrame(attendees_data)

# Sample data for companies_df
companies_data = {
    "company_url": ["company_1", "company_2", "company_3", "company_4", "company_5"],
    "company_name": ["TechCorp", "Innovatech", "OilCo", "GasCo", "MediHealth"],
    "company_industry": [
        "Technology",
        "Technology",
        "Oil & Gas",
        "Oil & Gas",
        "Healthcare",
    ],
    "company_revenue": [1000000, 1500000, 2000000, 2500000, 3000000],
    "company_country": ["USA", "USA", "USA", "Canada", "USA"],
}
companies_df = pd.DataFrame(companies_data)

# Sample data for contacts_df
contacts_data = {
    "company_url": ["company_1", "company_2", "company_3", "company_4", "company_5"],
    "office_city": ["San Francisco", "San Francisco", "Houston", "Calgary", "New York"],
    "office_country": ["USA", "USA", "USA", "Canada", "USA"],
    "office_address": [
        "123 Tech Lane",
        "456 Innovate St",
        "789 Oil Rd",
        "101 Gas Ave",
        "202 Medi Blvd",
    ],
    "office_email": [
        "contact@techcorp.com",
        "info@innovatech.com",
        "contact@oilco.com",
        "info@gasco.com",
        "contact@medihealth.com",
    ],
}
contacts_df = pd.DataFrame(contacts_data)

# Sample data for employees_df
employees_data = {
    "company_url": ["company_1", "company_2", "company_3", "company_4", "company_5"],
    "person_id": ["person_1", "person_2", "person_3", "person_4", "person_5"],
    "person_first_name": ["John", "Alice", "Robert", "Emily", "Michael"],
    "person_last_name": ["Doe", "Smith", "Johnson", "Davis", "Brown"],
    "person_email": [
        "john.doe@techcorp.com",
        "alice.smith@innovatech.com",
        "robert.johnson@oilco.com",
        "emily.davis@gasco.com",
        "michael.brown@medihealth.com",
    ],
    "person_city": ["San Francisco", "San Francisco", "Houston", "Calgary", "New York"],
    "person_country": ["USA", "USA", "USA", "Canada", "USA"],
    "person_seniority": ["Director", "Manager", "Director", "Manager", "Director"],
    "person_department": [
        "Engineering",
        "Product",
        "Operations",
        "Research",
        "Medical",
    ],
}
employees_df = pd.DataFrame(employees_data)

In [294]:
# Create an instance of the DataFrameFilter class and filter the data
filter_class = DataFrameFilter(
    events_df, attendees_df, companies_df, contacts_df, employees_df
)
filtered_events, filtered_companies, filtered_employees = filter_class.filter_data(
    # event_filters={"event_city": ["Houston"]},
    company_filters={"company_industry": ["Healthcare"]},
    # person_filters={"person_seniority": ["Director"]},
)
# Call the PrettyPrint class to print the filtered data.
PrettyPrint.print_dataframe(filtered_companies, "Filtered Companies")

PrettyPrint.print_dataframe(filtered_events, "Filtered Events")

PrettyPrint.print_dataframe(filtered_employees, "Filtered Employees")

Filtered Companies
+----+---------------+----------------+--------------------+-------------------+-------------------+
|    | company_url   | company_name   | company_industry   |   company_revenue | company_country   |
+====+===============+================+====================+===================+===================+
|  4 | company_5     | MediHealth     | Healthcare         |           3000000 | USA               |
+----+---------------+----------------+--------------------+-------------------+-------------------+
Filtered Events
+----+-------------+-----------------------+--------------------+---------------+-----------------+------------------+
|    | event_url   | event_name            | event_start_date   | event_city    | event_country   | event_industry   |
+====+=============+=======================+====================+===============+=================+==================+
|  0 | event_1     | Tech Summit           | 2024-09-10         | San Francisco | USA             | Tec

### Approach 2 implementation:

The second approach is to use the graph representaion. This will scale better but adds more complexity in the implementation. 

In [295]:
class GraphFilter:
    def __init__(self, uri, user, password):
        from neo4j import GraphDatabase

        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_constraints(self):
        # Ensure uniqueness of nodes with updated syntax
        with self.driver.session() as session:
            session.run("CREATE CONSTRAINT FOR (e:Event) REQUIRE e.event_url IS UNIQUE")
            session.run(
                "CREATE CONSTRAINT FOR (c:Company) REQUIRE c.company_url IS UNIQUE"
            )
            session.run(
                "CREATE CONSTRAINT FOR (p:Person) REQUIRE p.person_id IS UNIQUE"
            )

    def create_graph(self, events_df, attendees_df, companies_df, employees_df):
        with self.driver.session() as session:
            # Create Event nodes
            for _, row in events_df.iterrows():
                session.run(
                    """
                    MERGE (e:Event {event_url: $event_url})
                    SET 
                        e.event_name = $event_name,
                        e.event_start_date = $event_start_date,
                        e.event_city = $event_city,
                        e.event_country = $event_country,
                        e.event_industry = $event_industry
                    """,
                    event_url=row["event_url"],
                    event_name=row["event_name"],
                    event_start_date=row["event_start_date"],
                    event_city=row["event_city"],
                    event_country=row["event_country"],
                    event_industry=row["event_industry"],
                )

            # Create Company nodes
            for _, row in companies_df.iterrows():
                session.run(
                    """
                    MERGE (c:Company {company_url: $company_url})
                    SET 
                        c.company_name = $company_name,
                        c.company_industry = $company_industry,
                        c.company_revenue = $company_revenue,
                        c.company_country = $company_country
                    """,
                    company_url=row["company_url"],
                    company_name=row["company_name"],
                    company_industry=row["company_industry"],
                    company_revenue=row["company_revenue"],
                    company_country=row["company_country"],
                )

            # Create Person nodes and WORKS_AT relationships
            for _, row in employees_df.iterrows():
                session.run(
                    """
                    MERGE (p:Person {person_id: $person_id})
                    SET 
                        p.person_first_name = $person_first_name,
                        p.person_last_name = $person_last_name,
                        p.person_seniority = $person_seniority,
                        p.person_department = $person_department,
                        p.person_email = $person_email,
                        p.person_city = $person_city,
                        p.person_country = $person_country,
                        p.company_url = $company_url
                    """,
                    person_id=row["person_id"],
                    person_first_name=row["person_first_name"],
                    person_last_name=row["person_last_name"],
                    person_seniority=row["person_seniority"],
                    person_department=row["person_department"],
                    person_email=row["person_email"],
                    person_city=row["person_city"],
                    person_country=row["person_country"],
                    company_url=row["company_url"],
                )
                session.run(
                    """
                    MATCH (p:Person {person_id: $person_id}), (c:Company {company_url: $company_url})
                    MERGE (p)-[:WORKS_AT]->(c)
                    """,
                    person_id=row["person_id"],
                    company_url=row["company_url"],
                )

            # Create ATTENDED relationships
            for _, row in attendees_df.iterrows():
                session.run(
                    """
                    MATCH (e:Event {event_url: $event_url}), (c:Company {company_url: $company_url})
                    MERGE (c)-[:ATTENDED {relation_to_event: $company_relation_to_event}]->(e)
                    """,
                    event_url=row["event_url"],
                    company_url=row["company_url"],
                    company_relation_to_event=row["company_relation_to_event"],
                )

    def filter_data(self, event_filters=None, company_filters=None, person_filters=None):
        with self.driver.session() as session:
            query = """
            MATCH (e:Event)<-[:ATTENDED]-(c:Company)<-[:WORKS_AT]-(p:Person) 
            """
            where_clauses = []
            if event_filters:
                for key, _ in event_filters.items():
                    where_clauses.append(f"e.{key} IN $event_filters.{key}")
            if company_filters:
                for key, _ in company_filters.items():
                    where_clauses.append(f"c.{key} IN $company_filters.{key}")
            if person_filters:
                for key, _ in person_filters.items():
                    where_clauses.append(f"p.{key} IN $person_filters.{key}")
            if where_clauses:
                query += "WHERE " + " AND ".join(where_clauses) + " "
            query += "RETURN DISTINCT e, c, p"  # Use DISTINCT to eliminate duplicates

            result = session.run(
                query,
                event_filters=event_filters,
                company_filters=company_filters,
                person_filters=person_filters,
            )
            events, companies, people = [], [], []
            for record in result:
                events.append(record["e"])
                companies.append(record["c"])
                people.append(record["p"])

            return events, companies, people


In [296]:
graph_filter = GraphFilter("bolt://localhost:7687", "neo4j", "password")

# Note that if you are running for the first time then run the following lines of code to crate the initial graphs.
# Once the graphs are being created you can coimment out the following lines of codes.

# graph_filter.create_constraints()
graph_filter.create_graph(events_df, attendees_df, companies_df, employees_df)

In [297]:
filtered_events, filtered_companies, filtered_people = graph_filter.filter_data(
    # event_filters={"event_city": ["New York"]},
    person_filters={"person_seniority": ["Director"]},
    # company_filters={"company_industry": ["Healthcare"]},
)

In [298]:
PrettyPrint.print_node_data(filtered_events, "Filtered events")

PrettyPrint.print_node_data(filtered_companies, "Filtered companies")

PrettyPrint.print_node_data(filtered_people, "Filtered people")

graph_filter.close()

Filtered events
+----+------------------+---------------+-----------------+-----------------------+-------------+--------------------+
|    | event_industry   | event_city    | event_country   | event_name            | event_url   | event_start_date   |
+====+==================+===============+=================+=======================+=============+====================+
|  0 | Technology       | San Francisco | USA             | Tech Summit           | event_1     | 2024-09-10         |
+----+------------------+---------------+-----------------+-----------------------+-------------+--------------------+
|  1 | Oil & Gas        | Houston       | USA             | Oil & Gas Expo        | event_2     | 2024-10-15         |
+----+------------------+---------------+-----------------+-----------------------+-------------+--------------------+
|  2 | Healthcare       | New York      | USA             | Healthcare Conference | event_3     | 2024-11-20         |
+----+------------------+-------